# Project 3: Collaboration and Competition

### Algorithm Multi-Agent Deep Deterministic Policy Gradient for N agents (MADDPG)

1. Theory:

DDPG:

Basically, the actor learns to optimize the policy with the maximum state-action estimate for a given state, thus, following a detreministic policy, as opposed to a stochastic one. Again, this is due to maximizing the estimate for a state-action value. Thus, one needs to optimize:

$$argmax_{\mu}E_{S \sim D}[Q(s, \mu(s; \theta_{\mu}); \theta_{Q})]$$  

where $D=\{(s, a, r, s^{'}, d)_{0}, ..., (s, a, r, s^{'}, d)_{t}, ..., (s, a, r, s^{'}, d)_{T}\}$ is the buffer for the replay-memory.

On the other hand, the critic learns to approximate the optimal state-value function, $Q^{*}(s, a; \theta_{Q})$ where $a = \mu(s; \theta_{\mu})$ (detreministic policy that maximizes the probability of taking action $a$ given one is in state $s$). For stability, one uses a target function approximator and tries to learns how to maximize it:

$$argmin_{\theta_{Q}}E_{S,A \sim D}[(Q(S_{t}, A_{t}; \theta_{Q}) - y_{t})^{2}]$$

where $y_{t} = r(S_{t}, A_{t}) + \gamma * Q(S_{t + 1}, A_{t + 1}; \theta_{Q})$

The algorithm uses soft-updating for added stability (just as in the DQN algorithm):

$$\theta_{\mu}^{target} = \tau * \theta_{\mu} + (1 - \tau) * \theta_{\mu}^{target}$$
$$\theta_{Q}^{target} = \tau * \theta_{Q} + (1 - \tau) * \theta_{Q}^{target}$$



2. Algorithm:

**for** episode = 1 to num_episodes **do**
    
Initialize a random process N for action exploration
    
Receive initial observations, $x$ where $x=\{o_{0}, o_{1}, ..., o_{T}\}$
    
**for** t=1 to trajectory_size **do**
    
For each agent i, select $a_{i} = \mu(o_{i}; \theta_{\mu}) + N_{t}$

Execute $a=(a_{1}, ..., a_{N})$ and get reward $r$ and next observations $x^{'}$

Store $(x, a, r, x^{'})$ in $D$

$x = x^{'}$

**for** agent i=1 to N **do**

Sample batch $B=(x^{j}, a^{j}, r^{j}, x^{'j})$ from $D$

Update critic by $argmin_{\theta_{Q}}L(\theta_{Q})=\frac{1}{|B|}\sum_{j}(y^{j} - Q_{i}^{\mu}(x^{j}, a_{1}^{j}, ..., a_{N}^{j}; \theta_{Q}))^{2}$
 where $y^{j}=r_{i}^{j} + \gamma Q_{i}^{\mu^{'}}(x^{'j}, a_{1}^{'}, ..., a_{N}^{'})|_{a_{k}^{'}=\mu^{'}_{k}(o_{k}^{j};\theta_{\mu})}$

Update actor by  $argmax_{\theta_{\mu}}\frac{1}{|B|}\sum_{j}\nabla_{\theta_{\mu_{i}}}\mu_{i}(o_{i}^{j};\theta_{\mu})\nabla_{a_{i}}Q_{i}^{\mu_{i}}(x^{j}, a_{1}^{j}, ..., a_{N}^{j};\theta_{Q})|_{a_{i}=\mu_{i}(o_{i}^{j};\theta_{\mu})}$

**end for**

Soft-update for each agent $i$:

$\theta_{\mu}^{target} = \tau * \theta_{\mu} + (1 - \tau) * \theta_{\mu}^{target}$

$\theta_{Q}^{target} = \tau * \theta_{Q} + (1 - \tau) * \theta_{Q}^{target}$

**end for**

**end for**


3. Parameters:

$seed = 1$

$BatchSize = 1000$

$MemorySize = 1e6$

$\gamma = 0.95$

$lr_{actor} = 1e-3$

$lr_{critic} = 1e-3$


### Architecture

Actor: 3 fully connected layers (`nn.Linear`) with a tanh activation on the output to bound the output between -1 and 1. There is one batch normalization layer after the first fully connected one (`nn.BatchNorm1d`).

![alt](images/actor_architecture.png)

Critic: 3 fully connected layers where the second fully connected layer accepts as input (output_fc1 + action). There is one batch normalization layer after the first fully connected one.

![alt](images/critic_architecture.png)

All other activation functions are `F.leaky_relu`, in order to avoid dead neurons when learning.

### Training results

![alt](images/plot_scores.png)

### Ideas for future work

I would spawn few of those pairs in parallel processes and will have them update master weights. The architecture of these "nodes" will be in a distributed fashion, thus, perhaps getting rid of the replay memory and it might converge faster and better. I would aslo try a different algorithm such as AlphaGo or AlphaZero, which had great success.